In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from tqdm.notebook import tqdm
import pickle

In [2]:
url='https://leagueoflegends.fandom.com/wiki/List_of_champions'
r=requests.get(url)
soup=BeautifulSoup(r.text,'html.parser')

In [3]:
#Creating end part of each link for scraping

list_of_champs=[champion.text.replace(',','').split()[:2] for champion in soup.find_all('span',attrs={'data-skin':'Original'})]

two_names=['Aurelion','Dr.','Jarvan','Lee','Master','Miss','Tahm','Xin','Twisted']

#Turning two worded names into 1 string
list_of_champs=[champ[0]+' '+champ[1] if champ[0] in two_names else champ[0] for champ in list_of_champs]

#Choosing only distinct champions & then alphabetizing incase there are any duplicates.
list_of_champs=sorted(list(set(list_of_champs)))
list_of_champs

['Aatrox',
 'Ahri',
 'Akali',
 'Alistar',
 'Amumu',
 'Anivia',
 'Annie',
 'Aphelios',
 'Ashe',
 'Aurelion Sol',
 'Azir',
 'Bard',
 'Blitzcrank',
 'Brand',
 'Braum',
 'Caitlyn',
 'Camille',
 'Cassiopeia',
 "Cho'Gath",
 'Corki',
 'Darius',
 'Diana',
 'Dr. Mundo',
 'Draven',
 'Ekko',
 'Elise',
 'Evelynn',
 'Ezreal',
 'Fiddlesticks',
 'Fiora',
 'Fizz',
 'Galio',
 'Gangplank',
 'Garen',
 'Gnar',
 'Gragas',
 'Graves',
 'Hecarim',
 'Heimerdinger',
 'Illaoi',
 'Irelia',
 'Ivern',
 'Janna',
 'Jarvan IV',
 'Jax',
 'Jayce',
 'Jhin',
 'Jinx',
 "Kai'Sa",
 'Kalista',
 'Karma',
 'Karthus',
 'Kassadin',
 'Katarina',
 'Kayle',
 'Kayn',
 'Kennen',
 "Kha'Zix",
 'Kindred',
 'Kled',
 "Kog'Maw",
 'LeBlanc',
 'Lee Sin',
 'Leona',
 'Lissandra',
 'Lucian',
 'Lulu',
 'Lux',
 'Malphite',
 'Malzahar',
 'Maokai',
 'Master Yi',
 'Miss Fortune',
 'Mordekaiser',
 'Morgana',
 'Nami',
 'Nasus',
 'Nautilus',
 'Neeko',
 'Nidalee',
 'Nocturne',
 'Nunu',
 'Olaf',
 'Orianna',
 'Ornn',
 'Pantheon',
 'Poppy',
 'Pyke',
 'Qiyan

# Attempting with only 3 champions

In [4]:
#Saving the Models
def save_pickle(element,file):
    with open(file, 'wb') as handle:
        pickle.dump(element, handle, protocol=pickle.HIGHEST_PROTOCOL)

#Loading the models
def load_pickle(file):
    with open(file, 'rb') as handle:
        return pickle.load(handle)

In [5]:
driver = webdriver.Chrome(executable_path='/Users/Huski/Downloads/chromedriver')
driver.get('https://leagueoflegends.fandom.com/wiki/List_of_champions')

cookies_button=driver.find_element_by_class_name('_2o0B8MF50eAK1jv60jldUQ')
cookies_button.click()

In [7]:
health=[]
health_per_lvl=[]
health_regen=[]
health_regen_per_lvl=[]
ad=[]
ad_per_lvl=[]
armor=[]
armor_per_lvl=[]
mr=[]
mr_per_lvl=[]
attack_speed=[]
ms=[]
atk_range=[]
top3_attrs=[]


lst1=["Cho'Gath","Vel'Koz","Kha'Zix"]
lst2=["Kog'Maw","Rek'Sai"]

count=0

for champion in tqdm(list_of_champs):
    
    
    champion_button = driver.find_element_by_partial_link_text('{}'.format(champion))
    
    driver.execute_script("arguments[0].scrollIntoView();", champion_button)
    driver.execute_script("window.scrollBy(0, -100)")
    
    champion_button.click()
    
    time.sleep(5)
    
    soup=BeautifulSoup(driver.page_source,'html.parser')
    
    attributes=[]
    
    for x in soup.find_all('span',attrs={'class':'glossary'})[:3]:
        attributes.append(x.text)
    
    top3_attrs.append(attributes)
    
    
    # Source code is very inconsistent  
    name = champion
    if ' ' in champion:
        name=name.replace(' ','').replace('.','')
    elif champion=='Kled':
        name='Kled1'
    elif champion in lst1:
        name=name.replace("'",'').capitalize()
    elif champion in lst2:
        name=name.replace("'",'')
    
    health.append(soup.find('span',attrs={'id':"Health_{}".format(name)}).text)
    health_per_lvl.append(soup.find('span',attrs={'id':"Health_{}_lvl".format(name)}).text)
    health_regen.append(soup.find('span',attrs={'id':'HealthRegen_{}'.format(name)}).text)
    health_regen_per_lvl.append(soup.find('span',attrs={'id':'HealthRegen_{}_lvl'.format(name)}).text)

    ad.append(soup.find('span',attrs={'id':'AttackDamage_{}'.format(name)}).text)
    ad_per_lvl.append(soup.find('span',attrs={'id':'AttackDamage_{}_lvl'.format(name)}).text)

    armor.append(soup.find('span',attrs={'id':'Armor_{}'.format(name)}).text)
    armor_per_lvl.append(soup.find('span',attrs={'id':'Armor_{}_lvl'.format(name)}).text)

    mr.append(soup.find('span',attrs={'id':'MagicResist_{}'.format(name)}).text)
    mr_per_lvl.append(soup.find('span',attrs={'id':'MagicResist_{}_lvl'.format(name)}).text)

    attack_speed.append(soup.find('div',attrs={'data-source':'attack speed'}).text)

    ms.append(soup.find('span',attrs={'id':'MovementSpeed_{}'.format(name)}).text)

    atk_range.append(soup.find('span',attrs={'id':'AttackRange_{}'.format(name)}).text)
    
    count+=1
    if count%20==0:
        for stat in all_stats:
            save_pickle(stat,'first_{}_{}.pkl'.format(count,stat))

    driver.back()
    

OSError: [Errno 63] File name too long: "first_20_['Base AS0.651', 'Base AS0.668', 'Base AS0.625', 'Base AS0.625', 'Base AS0.736', 'Base AS0.625', 'Base AS0.579', 'Base AS0.64', 'Base AS0.658', 'Base AS0.625', 'Base AS0.625', 'Base AS0.625', 'Base AS0.625', 'Base AS0.625', 'Base AS0.644', 'Base AS0.681', 'Base AS0.644', 'Base AS0.647', 'Base AS0.625', 'Base AS0.638'].pkl"

In [9]:
for stat in all_stats:
    print()

AttributeError: 'list' object has no attribute 'str'

# Attemping with only 1 champion

In [ ]:
driver = webdriver.Chrome(executable_path='/Users/Huski/Downloads/chromedriver')
driver.get('https://leagueoflegends.fandom.com/wiki/List_of_champions')

cookies_button=driver.find_element_by_class_name('_2o0B8MF50eAK1jv60jldUQ')
cookies_button.click()

In [ ]:
champion="Cho'Gath"

In [ ]:
champion_button = driver.find_element_by_partial_link_text('{}'.format(champion))

time.sleep(2)

driver.execute_script("arguments[0].scrollIntoView();", champion_button)
driver.execute_script("window.scrollBy(0, -100)")

champion_button.click()

soup=BeautifulSoup(driver.page_source,'html.parser')

In [ ]:
health=[]
health_per_lvl=[]
health_regen=[]
health_regen_per_lvl=[]
ad=[]
ad_per_lvl=[]
armor=[]
armor_per_lvl=[]
mr=[]
mr_per_lvl=[]
attack_speed=[]
ms=[]
atk_range=[]
top3_attrs=[]

In [ ]:
name = champion
if ' ' in champion:
    name=name.replace(' ','').replace('.','')
elif champion=='Kled':
    name='Kled1'
elif champion in lst1:
    name=name.replace("'",'').capitalize()
elif champion in lst2:
    name=name.replace("'",'')

In [ ]:
name

In [ ]:

health.append(soup.find('span',attrs={'id':"Health_{}".format(name)}).text)
health_per_lvl.append(soup.find('span',attrs={'id':"Health_{}_lvl".format(name)}).text)
health_regen.append(soup.find('span',attrs={'id':'HealthRegen_{}'.format(name)}).text)
health_regen_per_lvl.append(soup.find('span',attrs={'id':'HealthRegen_{}_lvl'.format(name)}).text)

ad.append(soup.find('span',attrs={'id':'AttackDamage_{}'.format(name)}).text)
ad_per_lvl.append(soup.find('span',attrs={'id':'AttackDamage_{}_lvl'.format(name)}).text)

armor.append(soup.find('span',attrs={'id':'Armor_{}'.format(name)}).text)
armor_per_lvl.append(soup.find('span',attrs={'id':'Armor_{}_lvl'.format(name)}).text)

mr.append(soup.find('span',attrs={'id':'MagicResist_{}'.format(name)}).text)
mr_per_lvl.append(soup.find('span',attrs={'id':'MagicResist_{}_lvl'.format(name)}).text)

attack_speed.append(soup.find('div',attrs={'data-source':'attack speed'}).text)

ms.append(soup.find('span',attrs={'id':'MovementSpeed_{}'.format(name)}).text)

atk_range.append(soup.find('span',attrs={'id':'AttackRange_{}'.format(name)}).text)

driver.back()

In [ ]:
health

In [ ]:
for champion in list_of_champs:
    champion_button=driver.find_element_by_partial_link_text(champion)
    champion_button.click()
    
    soup=BeautifulSoup(driver.page_source,'html.parser')

In [ ]:
champion=driver.find_element_by_partial_link_text('Aatrox')
champion.click()

In [ ]:
soup=BeautifulSoup(driver.page_source,'html.parser')

In [ ]:
soup.find('span',attrs={'id':'Health_Ahri'}).text

In [ ]:
url='https://leagueoflegends.fandom.com/wiki/Ahri'
r=requests.get(url)
soup=BeautifulSoup(r.text,'html.parser')